# Calculate the Top N Stocks:
Top N by sharpe ratio and with a correlation of less than a specified `threshold`.

## 1. Download the data

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Example:
symbols = pd.read_csv('./data/ticker_names/sp500.csv', sep=',')
start_dt = datetime(2004,1,1)
end_dt = datetime(2014,1,1)

print(end_dt.weekday())

i = 1
data = {}
summary_data = {}

for symbol in symbols.Symbol:
    try:
        data[symbol] = yf.download(symbol, start=start_dt, end=end_dt, interval='1mo')['Adj Close']
        # volatility = returns.std()   # TODO: Try using negative volatility?        
        
    except Exception as e:
        print(f'Could not download ticker {symbol}. Error:\n{e}')
    
    i+= 1
    if i % 10 == 1:
        print(f'Downloading stock number {i}')

## 2. Drop keys without 10 years of data

In [ ]:
# Keep only tickers with full 10 years of data
for k in list(data.keys()):
    try:
        if data[k].index.min() > start_dt:
            data.pop(k)
    except:
        print(k)
        data.pop(k)

## 3. Calculate Sharpe

In [ ]:
summary_data = {}

for s, adj_close in data.items():
    returns = adj_close.pct_change()
    
    summary_data[s] = [returns.mean(), returns.std()]
    
summary_data_df = pd.DataFrame(summary_data).T
summary_data_df.columns = ['AVG_RETURNS','STDEV']
summary_data_df.dropna(inplace=True)
summary_data_df['SHARPE'] = summary_data_df.AVG_RETURNS / summary_data_df.STDEV

summary_data_df.sort_values('SHARPE', ascending=False)

## 4. Create data as dataframe and Pivot

In [ ]:
# TODO: Add check to ensure the correct dates are being pulled through 
dfs_with_keys = []

for symbol in summary_data_df.sort_values('SHARPE', ascending=False).index:
    df_copy = data[symbol].to_frame().pct_change()
    df_copy['TICKER'] = symbol
    dfs_with_keys.append(df_copy)

    # if len(dfs_with_keys) >= 50:
    #     break
        
merged_df = pd.concat(dfs_with_keys)
df_pivot = merged_df.pivot(columns='TICKER', values='Adj Close').dropna()
df_pivot.head()

## 5. Calculate Correlation

In [ ]:
df_corr = df_pivot.corr()
df_corr.head()

In [ ]:
threshold = 0.3
top_n_stocks = 20
final_tickers = []

for symbol in summary_data_df.sort_values('SHARPE', ascending=False).index:
    if len(final_tickers) > 0:
        # pick next stock - correlation to other tickers in the list
        corr_list = df_corr.loc[final_tickers, symbol]
        
        # if the correlation is above threshold, skip and pick the next
        if max(abs(corr_list)) < threshold:
            final_tickers.append(symbol) 
        
    else:
        final_tickers.append(symbol)

    if len(final_tickers) > top_n_stocks:
        break


In [ ]:
final_tickers

In [ ]:
# TODO: Add check to ensure the correct dates are being pulled through 
dfs_with_keys = []
# while len(dfs_with_keys) < 20:
for symbol in final_tickers:
    df_copy = data[symbol].to_frame() / data[symbol].iloc[0]
    df_copy['TICKER'] = symbol
    dfs_with_keys.append(df_copy)
        
merged_df = pd.concat(dfs_with_keys)

df_pivot = merged_df.pivot(columns='TICKER', values='Adj Close')

df_pivot.plot(figsize=(15,6))